<a href="https://www.kaggle.com/code/rishitjakharia/isruc-eda-modelling?scriptVersionId=214549012" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Importing

In [ ]:
import os
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from scipy.signal import butter, filtfilt

#### Setting Seed for reproducability

In [ ]:
import random
import os

seed_value = 42

random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

# Initial Graphs for class imbalance
----

```python
event_count = 0

for foldername, subfolders, filenames in os.walk('/kaggle/input/isruc-processed/dataset/Events'):
    if foldername.split('/')[-1] == 'Events':
        continue
    event_count += len(filenames)
    print(f"Event: '{foldername.split('/')[-1]}' has {len(filenames)} files.")


for foldername, subfolders, filenames in os.walk('/kaggle/input/isruc-processed/dataset/Non_Events'):
    if foldername.split('/')[-1] == 'Events':
        continue
    event_count += len(filenames)
    print(f"Event: '{foldername.split('/')[-1]}' has {len(filenames)} files.")

print(f"Total Event count: {event_count}")
```

Now, to maintain class balance when learning a sleep stage classifier and to have sufficient amount of non-event examples we shall sample the sum of all these.

Also, we are only looking at the cases where both Experts classify the event as a non-event.

In [ ]:
files = {
    'l on': 141,
    'ar': 25544,
    'mchg': 1530,
    'ch': 159,
    'l out': 218,
    'plm': 3402,
    'rem': 213,
    'awake': 4666,
    'oa': 1374,
    'ca': 362,
    'oh': 3652,
    'mh': 2707,
    'lm': 823,
    'non-events': 33531
}

# Prepare data for plotting
categories = list(files.keys())
values = list(files.values())

# Convert data to a DataFrame for seaborn
data = pd.DataFrame({
    'Category': categories,
    'Value': values
})

# Add a hue column for demonstration (e.g., splitting events into "Events" and "Non-events")
data['Type'] = ['Event' if cat != 'non-events' else 'Non-event' for cat in categories]

# Plotting with seaborn
plt.figure(figsize=(12, 6))
sns.barplot(data=data, x='Category', y='Value', hue='Type', palette='coolwarm')
plt.xlabel('Categories', fontsize=12)
plt.ylabel('Values', fontsize=12)
plt.title('Distribution of Event Data with Hue', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

There seems to be severe class imbalance, looking at the relevant labels now

In [ ]:
files = {
    'oa': 1374,
    'ca': 362,
    'oh': 3652,
    'mh': 2707,
    'ch': 159
}

# Prepare data for plotting
categories = list(files.keys())
values = list(files.values())

# Convert data to a DataFrame for seaborn
data = pd.DataFrame({
    'Category': categories,
    'Value': values
})

# Add a hue column for demonstration (e.g., splitting events into "Events" and "Non-events")
data['Type'] = ['Event' if cat != 'non-events' else 'Non-event' for cat in categories]

# Plotting with seaborn
plt.figure(figsize=(12, 6))
sns.barplot(data=data, x='Category', y='Value', hue='Type', palette='YlGnBu')
plt.xlabel('Categories', fontsize=12)
plt.ylabel('Values', fontsize=12)
plt.title('Distribution of Event Data with Hue', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Loading Data
----

## Sleep Stage Dataset

In [ ]:
class Normalization(object):
    def __init__(self, method="z-score", axis=1, epsilon=1e-8):
        """
        Args:
            method (str): Normalization method ('z-score' or 'min-max').
            axis (int): Axis along which to normalize (0: samples, 1: channels).
            epsilon (float): Small constant to avoid division by zero.
        """
        self.method = method
        self.axis = axis
        self.epsilon = epsilon

    def __call__(self, signal):
        """
        Apply normalization to the signal.

        Args:
            signal (torch.Tensor): Signal tensor of shape (n_channels, n_samples).
        
        Returns:
            torch.Tensor: Normalized signal tensor of the same shape.
        """
        signal_np = signal.numpy() if isinstance(signal, torch.Tensor) else signal

        if self.method == "z-score":
            mean = np.mean(signal_np, axis=self.axis, keepdims=True)
            std = np.std(signal_np, axis=self.axis, keepdims=True) + self.epsilon
            normalized_signal = (signal_np - mean) / std
        elif self.method == "min-max":
            min_val = np.min(signal_np, axis=self.axis, keepdims=True)
            max_val = np.max(signal_np, axis=self.axis, keepdims=True) + self.epsilon
            normalized_signal = (signal_np - min_val) / (max_val - min_val)
        else:
            raise ValueError("Unsupported normalization method. Choose 'z-score' or 'min-max'.")

        return torch.tensor(normalized_signal, dtype=torch.float32)

class BandpassFilter(object):
    def __init__(self, lowcut, highcut, fs, order=4, padding_value=0):
        """
        Args:
            lowcut (float): Low frequency cutoff (Hz).
            highcut (float): High frequency cutoff (Hz).
            fs (float): Sampling frequency (Hz).
            order (int): Order of the filter.
            padding_value (float): Value to use for padding short signals.
        """
        self.lowcut = lowcut
        self.highcut = highcut
        self.fs = fs
        self.order = order
        self.padding_value = padding_value
        
        # Design the filter
        nyquist = 0.5*fs
        self.b, self.a = butter(self.order, [self.lowcut/nyquist, self.highcut/nyquist], btype='band')

    def __call__(self, signal):
        """
        Apply the bandpass filter to each channel in the signal.

        Args:
            signal (tensor): Signal tensor of shape (n_channels, n_samples).
        
        Returns:
            tensor: Filtered signal tensor of the same shape.
        """
        filtered_signal = []
        
        # Apply the filter to each channel
        for i in range(signal.shape[1]):  # Iterate over channels
            channel_signal = signal[:, i]
            
            # Pad the signal if it's too short
            if len(channel_signal) <= self.order:
                padding_length = self.order - len(channel_signal) + 1
                channel_signal = np.pad(channel_signal, (padding_length, padding_length), mode='constant', constant_values=self.padding_value)
            
            # Apply the filter to the padded signal
            filtered_signal.append(filtfilt(self.b, self.a, channel_signal))

        # Convert back to tensor
        return torch.tensor(np.array(filtered_signal), dtype=torch.float32)

In [ ]:
class SleepStageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (str): Root directory containing all folders and signal files.
            transform (callable, optional): Optional transform to apply to the signals.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.data_info = self._prepare_file_list()

    def _prepare_file_list(self):
        """Scan the dataset directory and prepare a list of file paths and labels."""
        data_info = []
        for event_type in ['Events', 'Non_Events']:
            folder_path = os.path.join(self.root_dir, event_type)
            if os.path.isdir(folder_path):
                for subfolder_name in os.listdir(folder_path):
                    subfolder_path = os.path.join(folder_path, subfolder_name)
                    if os.path.isdir(subfolder_path):
                        for file_name in os.listdir(subfolder_path):
                            # Full file path
                            file_path = os.path.join(subfolder_path, file_name)
                            # Extraxt sleep stage label from filename
                            label = file_name.split('_')[3].replace('Stage', '')
                            
                            # Ignore errorneous labels
                            if label not in ['w', 'n1', 'n2', 'n3', 'r']:
                                continue
                            
                            data_info.append((file_path, label))
                            
        return data_info

    def __len__(self):
        return len(self.data_info)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Get file path and label
        file_path, label = self.data_info[idx]

        # converting label to one-hot
        label_to_one_hot = {
            'w':  [1,0,0,0,0],
            'n1': [0,1,0,0,0],
            'n2': [0,0,1,0,0],
            'n3': [0,0,0,1,0],
            'r':  [0,0,0,0,1],
        }
        label = label_to_one_hot[label]

        possible_columns = [
        ['X6', 'X7', 'X8', 'SaO2'],
        ['X6', 'X7', 'X8', 'SpO2'],
        ['29', '30', '31', 'SaO2'],
        ['29', '30', '31', 'SpO2']
        ]
        
        # Load signal data from .npy file
        signal = pd.read_csv(file_path)

        for columns in possible_columns:
            if all(col in signal.columns for col in columns):
                signal = signal[columns]
                break

        signal = torch.tensor(signal.values, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)

        if self.transform:
            for transform in self.transform:
                signal = transform(signal)

        return signal.T, label

In [ ]:
root_dir = '/kaggle/input/isruc-processed/dataset'
transform = [BandpassFilter(lowcut=0.2, highcut=0.5, fs=12.5), Normalization(method="z-score", axis=1)]

sleep_stage_dataset = SleepStageDataset(root_dir=root_dir, transform=transform)
sleep_stage_loader = DataLoader(sleep_stage_dataset, batch_size=1, shuffle=False)

count = 5
for signals, labels in sleep_stage_loader:
    print(signals.shape)
    print(labels)
    if count == 0:
        break
    count -= 1

## Events classification dataset

In [ ]:
class EventClassificationDataset(Dataset):
    def __init__(self, root_dir, transform=None, label_mapping=None, included_classes=None):
        """
        Args:
            root_dir (str): Root directory containing all folders and signal files.
            transform (callable, optional): Optional transform to apply to the signals.
            included_classes (list, optional): List of classes to include. If None, include all classes.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.included_classes = included_classes
        self.label_mapping = label_mapping
        self.data_info = self._prepare_file_list()

    def _prepare_file_list(self):
        """Scan the dataset directory and prepare a list of file paths and labels."""
        data_info = []

        # Process Event files
        upper_limit_per_event = {
            'oa': 150,
            'ca': 150,
            'oh': 150,
            'mh': 150,
            'ch': 150
        }
        
        current_count = {
            'oa': 0,
            'ca': 0,
            'oh': 0,
            'mh': 0,
            'ch': 0
        }
        upper_limit_to_no_events = 300
        
        event_path = os.path.join(self.root_dir, 'Events')
        if os.path.isdir(event_path):
            for label_folder in os.listdir(event_path):
                if self.included_classes and label_folder not in self.included_classes:
                    continue

                label_path = os.path.join(event_path, label_folder)
                if os.path.isdir(label_path):
                    for file_name in os.listdir(label_path):
                        # Full file path
                        file_path = os.path.join(label_path, file_name)

                        # Append (file_path, label) tuple
                        if upper_limit_per_event[label_folder] != -1 and current_count[label_folder] == upper_limit_per_event[label_folder]:
                            continue
                        else:
                            current_count[label_folder] += 1
                        
                        data_info.append((file_path, label_folder))

        # Process Non Event files
        non_event_path = os.path.join(self.root_dir, 'Non_Events')
        if os.path.isdir(non_event_path):
            for subfolder_name in os.listdir(non_event_path):
                subfolder_path = os.path.join(non_event_path, subfolder_name)
                if os.path.isdir(subfolder_path):
                    for file_name in os.listdir(subfolder_path):
                        # Full file path
                        file_path = os.path.join(subfolder_path, file_name)

                        # Label for non-events is "no_event"
                        if not self.included_classes or 'no_event' in self.included_classes:
                            data_info.append((file_path, 'no_event'))

                        if upper_limit_to_no_events == 0:
                            break
                        upper_limit_to_no_events -= 1

        return data_info

    def __len__(self):
        return len(self.data_info)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Get file path and label
        file_path, label = self.data_info[idx]

        # Convert label to one-hot encoding
        unique_labels = self.included_classes if self.included_classes else ['ar', 'awake', 'ca', 'ch', 'l on', 'l out', 
                                                                             'lm', 'mchg', 'mh', 'oa', 'oh', 'plm', 'rem', 
                                                                             'no_event']
        label_to_one_hot = {label: [1 if i == idx else 0 for i in range(len(unique_labels))] 
                            for idx, label in enumerate(unique_labels)}
        label = label_to_one_hot[label]

        possible_columns = [
        ['X6', 'X7', 'X8', 'SaO2'],
        ['X6', 'X7', 'X8', 'SpO2'],
        ['29', '30', '31', 'SaO2'],
        ['29', '30', '31', 'SpO2']
        ]
        
        signal = pd.read_csv(file_path)
    
        for columns in possible_columns:
            if all(col in signal.columns for col in columns):
                signal = signal[columns]
                break
        
        signal = torch.tensor(signal.values, dtype=torch.float32)
        if self.label_mapping:
            label = self.label_mapping[np.argmax(label)]
        label = torch.tensor(label, dtype=torch.long)
        
        if self.transform:
            for transform in self.transform:
                signal = transform(signal)

        return signal.T, label

In [ ]:
root_dir = '/kaggle/input/isruc-processed/dataset'
included_classes = ['ca', 'oa', 'oh', 'mh', 'ch', 'no_event']
transform = [BandpassFilter(lowcut=0.2, highcut=0.5, fs=12.5), Normalization(method='z-score', axis=1)]

event_classification_dataset = EventClassificationDataset(root_dir=root_dir, transform=transform, included_classes=included_classes)
event_classification_loader = DataLoader(event_classification_dataset, batch_size=1, shuffle=True)

count = 3
for signals, labels in event_classification_loader:
    print(included_classes[np.argmax(labels)])
    print('-'*100)
    if count == 0:
        break
    count -= 1

# Modelling
----

## Loss Function

In [ ]:
def kl_divergence_loss(preds, targets):
    targets = targets.float()
    
    preds = F.softmax(preds, dim=-1)
    targets = F.softmax(targets, dim=-1)
    loss = F.kl_div(preds.log(), targets, reduction='batchmean')
    return loss

def mse_loss(preds, targets):
    # Ensure that the predictions and targets are one-hot encoded
    preds = F.softmax(preds, dim=-1)
    return F.mse_loss(preds, targets)

# Focal Loss
def focal_loss(preds, targets, alpha=0.25, gamma=2.0):
    """
    Focal Loss for multi-class classification.
    
    Arguments:
    preds -- the raw logits from the model (shape: [batch_size, num_classes])
    targets -- the true labels (shape: [batch_size])
    alpha -- balancing factor for class imbalances (default is 0.25)
    gamma -- focusing parameter (default is 2.0)
    
    Returns:
    loss -- the computed focal loss
    """
    # Apply softmax to get probabilities
    preds = F.softmax(preds, dim=-1)
    
    # Convert targets to one-hot encoding
    targets_one_hot = targets
    
    # Cross-entropy loss
    ce_loss = F.cross_entropy(preds, targets, reduction='none')
    
    # Get the predicted probability for the correct class
    pt = torch.exp(-ce_loss)
    
    # Compute focal loss
    focal_loss = alpha * (1 - pt) ** gamma * ce_loss
    
    return focal_loss.mean()

## Optimizer and Learning Rate

In [ ]:
def warmup_lr(epoch, step_size, warmup_epochs=5, gamma=0.1):
    if epoch < warmup_epochs:
        return (epoch + 1) / warmup_epochs
    
    else:
        epoch_since_warmup = epoch - warmup_epochs
        return gamma ** (epoch_since_warmup // step_size)

WARMUP_EPOCHS = 14
GAMMA = 0.1
STEP_SIZE = 20

## Plotting Train and Test Acc and Loss

In [ ]:
# Plot history
def plot_history(history):
    epochs = range(1, len(history['train_loss']) + 1)

    plt.figure(figsize=(12, 4))

    # Loss plot
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], label='Train Loss')
    plt.plot(epochs, history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss History')

    # Accuracy plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['train_acc'], label='Train Accuracy')
    plt.plot(epochs, history['val_acc'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Accuracy History')

    plt.show()

## Confusion Matrix Function

In [ ]:
def plot_confusion_matrix(model, val_loader, class_names, device="cuda"):
    """
    Evaluates the model and plots a confusion matrix with a custom color map.

    Arguments:
    - model: Trained PyTorch model.
    - val_loader: DataLoader for validation dataset.
    - class_names: List of class names for the confusion matrix.
    - device: 'cuda' or 'cpu'.
    """
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)  # Predicted classes
            
            # If labels are one-hot encoded, convert them to class indices
            if len(labels.shape) > 1 and labels.size(1) > 1:
                labels = torch.argmax(labels, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Compute confusion matrix
    cm = confusion_matrix(all_labels, all_preds, labels=np.arange(len(class_names)))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

    # Plot the confusion matrix with customization
    fig, ax = plt.subplots(figsize=(10, 8))  # Set larger figure size
    disp.plot(cmap="YlGnBu", ax=ax, colorbar=True)  # Use "cividis" colormap and add colorbar
    plt.title("Confusion Matrix", fontsize=16)
    plt.xticks(fontsize=12, rotation=45)
    plt.yticks(fontsize=12)
    plt.show()

## Training Function

In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import LeaveOneOut
from collections import defaultdict


def train_model(
    model,
    train_loader,
    val_loader,
    criterion=None,
    optimizer=None,
    scheduler=None,
    epochs=20,
    device="cpu",
    early_stopping=False,
    patience=20,
    min_delta=1e-3,
    save_path="best_model.pth",
    verbose=True
):
    """
    Train a PyTorch model with optional early stopping and learning rate scheduling.

    Args:
        model (torch.nn.Module): The model to train.
        train_loader (DataLoader): DataLoader for the training dataset.
        val_loader (DataLoader): DataLoader for the validation dataset.
        criterion (callable, optional): Loss function. Default is F.mse_loss.
        optimizer (torch.optim.Optimizer): Optimizer for training.
        scheduler (callable, optional): Learning rate scheduler.
        epochs (int, optional): Number of training epochs. Default is 20.
        device (str, optional): Device to train on ("cpu" or "cuda"). Default is "cpu".
        early_stopping (bool, optional): Whether to enable early stopping. Default is False.
        patience (int, optional): Patience for early stopping. Default is 20.
        min_delta (float, optional): Minimum delta for early stopping. Default is 1e-3.
        save_path (str, optional): Path to save the best model. Default is 'best_model.pth'.
        verbose (bool, optional): Whether to print training progress. Default is True.

    Returns:
        dict: History of training and validation loss and accuracy.
    """
    history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}

    best_loss = np.inf
    best_acc = -np.inf
    patience_counter = 0
    best_model_state = None

    if criterion is None:
        criterion = F.mse_loss

    for epoch in range(epochs):
        model.train()
        train_loss, train_correct = 0.0, 0

        # Training phase
        with tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs} - Training", unit="batch", disable=not verbose) as tepoch:
            for inputs, targets in tepoch:
                inputs = inputs.to(device)
                targets = targets.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets.float())
                loss.backward()
                optimizer.step()

                train_loss += loss.item() * inputs.size(0)
                train_correct += (outputs.argmax(1) == targets.argmax(1)).sum().item()

                if verbose:
                    tepoch.set_postfix(loss=loss.item())

        if scheduler is not None:
            scheduler.step()

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct / len(train_loader.dataset)

        # Validation phase
        model.eval()
        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            with tqdm(val_loader, desc=f"Epoch {epoch + 1}/{epochs} - Validation", unit="batch", disable=not verbose) as vepoch:
                for inputs, targets in vepoch:
                    inputs = inputs.to(device)
                    targets = targets.to(device)

                    outputs = model(inputs)
                    loss = criterion(outputs, targets.float())
                    val_loss += loss.item() * inputs.size(0)
                    val_correct += (outputs.argmax(1) == targets.argmax(1)).sum().item()

                    if verbose:
                        vepoch.set_postfix(loss=loss.item())

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / len(val_loader.dataset)

        # Save history
        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)

        if verbose:
            print(
                f"Epoch {epoch + 1}/{epochs} - "
                f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} - "
                f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}"
            )

        # Save the best model
        if best_acc < val_acc - min_delta:
            best_loss = val_loss
            best_acc = val_acc
            patience_counter = 0
            best_model_state = model.state_dict()
            if verbose:
                print(f"New best model found at epoch {epoch + 1} with val_loss: {val_loss:.4f}")
        else:
            patience_counter += 1

        # Early stopping
        if early_stopping and patience_counter >= patience:
            if verbose:
                print("Early stopping triggered!")
            break

    # Save the best model at the end of training
    if best_model_state is not None:
        torch.save(best_model_state, save_path)
        if verbose:
            print(f"Best model saved to {save_path} with val_loss: {best_loss:.4f} and val_acc: {best_acc:.4f}")

    return history

def train_with_loocv(
    model_class,
    model_params,
    dataset,
    batch_size,
    optimizer_class,
    optimizer_params,
    shuffle=True,
    criterion=None,
    scheduler_class=None,
    scheduler_params=None,
    epochs=20,
    device="cpu",
    early_stopping=False,
    patience=20,
    min_delta=1e-3,
    save_path="best_model.pth",
    verbose=True
):
    """
    Train a PyTorch model using Leave-One-Out Cross-Validation (LOOCV), grouped by patient-session.

    Args:
        model_class (torch.nn.Module): The model class to instantiate for each fold.
        model_params (dict): Parameters for the model initialization.
        dataset (Dataset): The dataset to perform LOOCV on.
        batch_size (int): The batch size for the train loader.
        optimizer_class (torch.optim.Optimizer): Optimizer class.
        optimizer_params (dict): Parameters to initialize the optimizer.
        shuffle (bool): Whether to shuffle the train dataset or not. Default is True.
        criterion (callable, optional): Loss function. Default is `mse_loss`.
        scheduler_class (callable, optional): Learning rate scheduler class. Default is None.
        scheduler_params (dict, optional): Parameters for the scheduler. Default is None.
        epochs (int, optional): Number of epochs for training. Default is 20.
        device (str, optional): Device to train on ("cpu" or "cuda"). Default is "cpu".
        early_stopping (bool, optional): Whether to enable early stopping. Default is False.
        patience (int, optional): Patience for early stopping. Default is 20.
        min_delta (float, optional): Minimum delta for early stopping. Default is 1e-3.
        save_path (str, optional): Path to save the best model for each fold. Default is 'best_model.pth'.
        verbose (bool, optional): Whether to print training progress. Default is True.

    Returns:
        list: A list of histories for each fold.
    """
    # Group data by patient-session based on filename
    patient_session_groups = defaultdict(list)

    for idx, (_, label) in enumerate(dataset.data_info):
        filename = dataset.data_info[idx][0]
        filename = filename.split("/")[-1]
        patient_session_key = "_".join(filename.split("_")[:2])  # e.g., "S1_p_1"
        patient_session_groups[patient_session_key].append(idx)

    # Prepare LOOCV splits based on patient-session groups
    group_keys = list(patient_session_groups.keys())
    loo = LeaveOneOut()
    histories = []

    for fold, (train_group_idx, val_group_idx) in enumerate(loo.split(group_keys)):
        if verbose:
            print(f"\nFold {fold + 1}/{len(group_keys)}")

        # Get train and validation indices from groups
        train_idx = [idx for group_idx in train_group_idx for idx in patient_session_groups[group_keys[group_idx]]]
        val_idx = [idx for group_idx in val_group_idx for idx in patient_session_groups[group_keys[group_idx]]]

        train_subset = torch.utils.data.Subset(dataset, train_idx)
        val_subset = torch.utils.data.Subset(dataset, val_idx)

        train_loader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=shuffle)
        val_loader = torch.utils.data.DataLoader(val_subset, batch_size=batch_size, shuffle=False)

        model = model_class(**model_params).to(device)
        optimizer = optimizer_class(model.parameters(), **optimizer_params)
        scheduler = None
        if scheduler_class is not None:
            scheduler = scheduler_class(optimizer, **scheduler_params)
        
        # Train the model for the current fold
        history = train_model(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            criterion=criterion,
            optimizer=optimizer,
            scheduler=scheduler,
            epochs=epochs,
            device=device,
            early_stopping=early_stopping,
            patience=patience,
            min_delta=min_delta,
            save_path=f"{save_path}_fold_{fold+1}.pth",
            verbose=verbose,
        )

        # Append history for this fold
        histories.append(history)

    return histories

## Models
----
### Bi-LSTM + Attention

In [ ]:
class AdaptiveRNN(nn.Module):
    def __init__(self, n_timesteps, n_features, n_outputs):
        super(AdaptiveRNN, self).__init__()

        # Bi-directional LSTM layer
        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=256,
            batch_first=True,
            bidirectional=True
        )
        self.batch_norm = nn.BatchNorm1d(256 * 2)
        self.batch_norm1 = nn.BatchNorm1d(128)

        # Attention mechanism
        self.attention_dense = nn.Linear(256 * 2, 1)
        
        # Fully connected layers
        self.fc1 = nn.Linear(256 * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, n_outputs)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # LSTM output
        lstm_out, _ = self.lstm(x)  # lstm_out shape: (batch_size, timesteps, 128*2)
        
        batch_size, n_timesteps, n_features = lstm_out.size()
        lstm_out = lstm_out.reshape(batch_size * n_timesteps, n_features)
        lstm_out = self.batch_norm(lstm_out)
        lstm_out = lstm_out.reshape(batch_size, n_timesteps, n_features)
        
        # Attention mechanism
        attention_scores = torch.sigmoid(self.attention_dense(lstm_out))  # (batch_size, timesteps, 1)
        attention_scores = lstm_out * attention_scores  # Weighted timesteps

        # Global average pooling
        pooled = attention_scores.mean(dim=1)
        
        # Fully connected layers
        x = F.relu(self.fc1(pooled))
        x = self.batch_norm1(x)
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return F.softmax(x, dim=1)

In [ ]:
# Define the model parameters
n_timesteps = 30 * 12.5
n_features = 4
n_outputs = 3

# Create the model
model = AdaptiveRNN(n_timesteps=n_timesteps, n_features=n_features, n_outputs=n_outputs).to(device)
summary(model)

```python
LEARNING_RATE = 0.001
BATCH_SIZE = 16
T_MAX = 50
ETA_MIN = 1e-6
EPOCHS = 10
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model_class = AdaptiveRNN
model_params = {"n_timesteps": 375, "n_features": 4, "n_outputs": 6}

# Call train_with_loocv
# history = train_with_loocv(
#     model_class=model_class,
#     model_params=model_params,
#     dataset=event_classification_dataset,
#     batch_size=BATCH_SIZE,
#     criterion=kl_divergence_loss,
#     optimizer_class=torch.optim.Adam,
#     optimizer_params={"lr": LEARNING_RATE},
#     scheduler_class=torch.optim.lr_scheduler.CosineAnnealingLR,
#     scheduler_params={"T_max": T_MAX, "eta_min": ETA_MIN},
#     epochs=EPOCHS,
#     device=DEVICE,
#     early_stopping=False,  # Set to True if you want early stopping
#     patience=20,  # Adjust patience if early stopping is enabled
#     verbose=True
# )
```

In [ ]:
root_dir = '/kaggle/input/isruc-processed/dataset'
included_classes = ['ca', 'oa', 'oh', 'mh', 'ch', 'no_event']
transform = [BandpassFilter(lowcut=0.2, highcut=0.5, fs=12.5), Normalization(method='min-max', axis=1)]

# label_mapping = {
#     'ca': 'apnea',
#     'oa': 'apnea',
#     'ch': 'apnea',
#     'oh': 'hypopnea',
#     'mh': 'hypopnea',
#     'no_event': 'no_event'
# }

label_mapping = [[1,0,0], [1,0,0], [1,0,0],
                 [0,1,0], [0,1,0],
                 [0,0,1]
                ]

# Create an instance of the SleepStageDataset
event_classification_dataset = EventClassificationDataset(root_dir=root_dir, transform=transform, label_mapping=label_mapping, included_classes=included_classes)

patient_session_groups = defaultdict(list)

for idx, (_, label) in enumerate(event_classification_dataset.data_info):
    filename = event_classification_dataset.data_info[idx][0]
    filename = filename.split("/")[-1]
    patient_session_key = "_".join(filename.split("_")[:2])  # e.g., "S1_p_1"
    patient_session_groups[patient_session_key].append(idx)

group_keys = list(patient_session_groups.keys())
train_group_keys, test_group_keys = train_test_split(
    group_keys, test_size=0.1, random_state=42
)

# Get train and test indices from groups
train_idx = [idx for key in train_group_keys for idx in patient_session_groups[key]]
test_idx = [idx for key in test_group_keys for idx in patient_session_groups[key]]

# Create train and test subsets
train_subset = torch.utils.data.Subset(event_classification_dataset, train_idx)
test_subset = torch.utils.data.Subset(event_classification_dataset, test_idx)

# Create data loaders
batch_size = 256
train_loader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_subset, batch_size=batch_size, shuffle=False)

print(f"Number of training samples: {len(train_subset)}")
print(f"Number of testing samples: {len(test_subset)}")

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-6)

history = train_model(
    model = model,
    train_loader=train_loader,
    val_loader=test_loader,
    criterion=focal_loss,
    optimizer=optimizer,
    scheduler=scheduler,
    epochs=5,
    device=device,
    early_stopping=False,
    patience=20,
    min_delta=1e-3,
    save_path="best_model.pth",
    verbose=True
)

In [ ]:
plot_confusion_matrix(model, test_loader, class_names=['apnea', 'hypopnea', 'no events'])

### Log
----

1) **LOOCV**:

Bi-LSTM + Attention Model + Cross Entropy + Complete Dataset with undersampled `no_events` -> 50% Train, 28% Val

Bi-LSTM + Attention Model + KL Div Loss + Complete Dataset with undersampled `no_events` -> 50% Train, 37% Val

Bi-LSTM + Attention Model + KL Div Loss + UnderSampled Dataset -> 57.57% Train, 28.81% Val

2) **Train-Test Split using PatientID**:

Bi-LSTM + Attention Model + KL Div Loss + UnderSampled Dataset -> 52.41% Train, 51.58% Val